# 数据标签

In [ ]:
import datetime
import pandas as pd

label_dict = {
    'A': 'NF',
    'B': 'NF',
    'C': 'C',
    'M': 'M',
    'X': 'X'
}

def label(fl_class:str):
    return label_dict.get(fl_class[0] if fl_class else '', 'NF')

if __name__ == '__main__':
    magLabels = []

    # 预报时间限制
    predictLimit = 24 

    # 磁场数据
    magFiles = pd.read_csv("mag files.csv")

    # 耀斑数据
    flares = pd.read_csv("GOES Flare Catalogue.csv")
    flares['start'] = pd.to_datetime(flares['start'])
    flares['peak'] = pd.to_datetime(flares['peak'])
    flares['end'] = pd.to_datetime(flares['end'])

    # 标注
    for i in range(len(magFiles)):
        file = magFiles['file'][i]
        indexTime = file.split(".")[-2]
        indexTime = datetime.datetime.strptime(indexTime, "%Y%m%d_%H%M%S_TAI")

        # 查找峰值时间在未来时间窗口内的事件
        events = flares[(indexTime <= flares['peak']) & 
                        (flares['peak'] < indexTime + datetime.timedelta(hours=predictLimit))]
        
        # 若事件为空
        if events.empty: 
            magLabels.append([file, indexTime, None, label(None)])
       
        # 若事件不为空
        if not events.empty:
            # 按 'fl_class' 排序并获取强度等级最大事件
            maxEvent = events.loc[events['fl_class'].idxmax()]
            maxClass = maxEvent['fl_class']
            magLabels.append([file, indexTime, maxClass, label(maxClass)])

    magLabels = pd.DataFrame(magLabels, columns=['file', 'date', 'fl_class', 'label'])
    magLabels.to_csv("labels.csv", index = False)